In [7]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )

def embedding_conversion(li,embedding_dim=1024):
    flat_embedding=li
    if len(flat_embedding) % embedding_dim != 0:
        raise ValueError("Flat embedding length is not divisible by embedding dimension.")
    
    num_vectors = len(flat_embedding) // embedding_dim
    section_embedding_2d = [
        flat_embedding[i * embedding_dim: (i + 1) * embedding_dim]
        for i in range(num_vectors)
    ]
    return section_embedding_2d

core_name = "mycore"
solr_url = f"http://localhost:8983/solr/{core_name}/select"
initial_params = {
    "q": "*:*",
    "wt": "json",
    "rows": 0
}
response = requests.get(solr_url, params=initial_params)
total_docs = response.json()["response"]["numFound"]
params = {
    "q": "*:*",
    "wt": "json",
    "rows": total_docs
}
response = requests.get(solr_url, params=params)
docs = response.json()["response"]["docs"]
dic={}
docs1=docs
for i in range(len(docs)):
    li=[docs[i]['class'][0],docs[i]['text'][0],embedding_conversion(docs[i]["section_embedding"]),docs[i]['total_embedding']]
    dic[i]=li

import faiss
from sklearn.metrics.pairwise import euclidean_distances,cosine_similarity
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
)
def context_provider(dic,query):
    idx=''
    query=model.embed_query(query)
    query_emb=np.array([query],dtype='float32')
    sim=0
    for i in dic:
        tot_emb=np.array([dic[i][-1]])

        cos_sim = cosine_similarity(query_emb, tot_emb)
        print(cos_sim)
        if(cos_sim>sim):
            sim=cos_sim
            idx=i
    text,sec_emb=dic[idx][1],np.array(dic[idx][2],dtype='float32')
    faiss.normalize_L2(sec_emb)
    db = faiss.IndexFlatIP(sec_emb.shape[1])
    db.add(sec_emb)
    D, I = db.search(query_emb, k=8)
    context=""
    text_split=text_splitter.split_text(text)
    # print(I)
    for i in I[0]:
        context+=text_split[i]
        context+='\n'
    return context

query=input("What is your question:: ")
print(context_provider(dic,query))

[[0.75993221]]
[[0.7765833]]
[[0.77280991]]
--- Page 1 ---
Confidential document 
 
 
 
Performance 
Management 
Policy 
Document No : 
Page No 
: 1 of 4 
Revision No. : 
Effective Date : 
Document Type: 
Master 
Performance Management Policy 
 
 
DOCUMENT AUTHORIZATION 
 
 
Name 
Signature 
Date 
Prepared by 
Kundan Kashyap 
 
01-Jan-2025 
Reviewed by 
Ritesh Kumar 
 
01-Jan-2025 
Approved by 
Ritesh Kumar 
 
01-Jan-2025 
 
REVISION HISTORY: 
 
Rev. No. 
Date 
Change Description 
Approved by 
Signature 
Initial Release
--- Page 3 ---
Confidential document 
 
 
 
Performance 
Management 
Policy 
Document No : 
Page No 
: 3 of 4 
Revision No. : 
Effective Date : 
Document Type: 
Master 
Performance Management Policy 
 
6. Development Plans: 
• 
Based on performance reviews, PIP plans will be created for each employee. 
• 
Plans may include training, mentoring, job rotation, or other development activities. 
• 
Progress on development plans will be reviewed regularly. 
7. Recognition and

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

device='cuda'
model_name="mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
tokenizer=AutoTokenizer.from_pretrained(model_name,)
if tokenizer.pad_token==None:
    tokenizer.pad_token=tokenizer.eos_token
model=AutoModelForCausalLM.from_pretrained(model_name,
                                           torch_dtype=torch.bfloat16,
                                           quantization_config=bnb_config).to(device)

message=[
    {
        "role":"system",
        "content":"you are a my sql query generater. Generate query based on user prompt"
    },
    {
        "role":"user",
        "content":"i want to see how many people were absent yesterday [table:- employees,metadata: 'emp id','leaves','working hrs']"
    }
]


prompt=tokenizer.apply_chat_template(message, tokenize=False)
inputs=tokenizer(prompt,return_tensors='pt').to(device)

out=model.generate(inputs['input_ids'],max_length=500)

print(tokenizer.decode(out[0]))
